In [20]:
import numpy as np
import pandas as pd

In [3]:
sagpred2 = np.load('sagittal_preds.npy')

In [4]:
corpred2 = np.load('coronal_preds.npy')

In [5]:
axipred2 = np.load('axial_preds.npy')

In [11]:
y = np.load('gt.npy')

In [6]:
sag_abn_acc = 0.783
sag_acl_acc = 0.642
sag_men_acc = 0.592

cor_abn_acc = 0.792
cor_acl_acc = 0.583
cor_men_acc = 0.550

axi_abn_acc = 0.817
axi_acl_acc = 0.692
axi_men_acc = 0.575

In [7]:
w_abn = [np.log(sag_abn_acc/(1-sag_abn_acc)),
         np.log(cor_abn_acc/(1-cor_abn_acc)),
         np.log(axi_abn_acc/(1-axi_abn_acc))]

w_acl = [np.log(sag_acl_acc/(1-sag_acl_acc)),
         np.log(cor_acl_acc/(1-cor_acl_acc)),
         np.log(axi_acl_acc/(1-axi_acl_acc))]

w_men = [np.log(sag_men_acc/(1-sag_men_acc)),
         np.log(cor_men_acc/(1-cor_men_acc)),
         np.log(axi_men_acc/(1-axi_men_acc))]

In [8]:
w_abn = w_abn / np.sum(w_abn)
w_acl = w_acl / np.sum(w_acl)
w_men = w_men / np.sum(w_men)

In [9]:
abn_pred = []
acl_pred = []
men_pred = []

for i in range(120):
    abn_pred.append(sagpred2[i,0]*w_abn[0]+corpred2[i,0]*w_abn[1]+axipred2[i,0]*w_abn[2])
    acl_pred.append(sagpred2[i,1]*w_acl[0]+corpred2[i,1]*w_acl[1]+axipred2[i,1]*w_acl[2])
    men_pred.append(sagpred2[i,2]*w_men[0]+corpred2[i,2]*w_men[1]+axipred2[i,2]*w_men[2])

In [10]:
abn_pred = np.array(abn_pred).reshape(120,1)
acl_pred = np.array(acl_pred).reshape(120,1)
men_pred = np.array(men_pred).reshape(120,1)
pred = np.append(np.append(abn_pred,acl_pred,axis=1),men_pred,axis=1)

In [13]:
#util_wk2

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, f1_score
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_auc_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

def TP(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == True) & (y == 1))


def TN(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == False) & (y == 0))


def FN(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == False) & (y == 1))


def FP(y, pred, th=0.5):
    pred_t = (pred > th)
    return np.sum((pred_t == True) & (y == 0))

def get_accuracy(y, pred, th=0.5):
    tp = TP(y,pred,th)
    fp = FP(y,pred,th)
    tn = TN(y,pred,th)
    fn = FN(y,pred,th)
    
    return (tp+tn)/(tp+fp+tn+fn)

def get_prevalence(y):
    return np.sum(y)/y.shape[0]

def sensitivity(y, pred, th=0.5):
    tp = TP(y,pred,th)
    fn = FN(y,pred,th)
    
    return tp/(tp+fn)

def specificity(y, pred, th=0.5):
    tn = TN(y,pred,th)
    fp = FP(y,pred,th)
    
    return tn/(tn+fp)

def get_ppv(y, pred, th=0.5):
    tp = TP(y,pred,th)
    fp = FP(y,pred,th)
    
    return tp/(tp+fp)

def get_npv(y, pred, th=0.5):
    tn = TN(y,pred,th)
    fn = FN(y,pred,th)
    
    return tn/(tn+fn)


def get_performance_metrics(y, pred, class_labels, tp=TP,
                            tn=TN, fp=FP,
                            fn=FN,
                            acc=get_accuracy, prevalence=get_prevalence, 
                            spec=specificity,sens=sensitivity, ppv=get_ppv, 
                            npv=get_npv, auc=roc_auc_score, f1=f1_score,
                            thresholds=[]):
    if len(thresholds) != len(class_labels):
        thresholds = [.5] * len(class_labels)

    columns = ["Injury", "TP", "TN", "FP", "FN", "Accuracy", "Prevalence",
               "Sensitivity",
               "Specificity", "PPV", "NPV", "AUC", "F1", "Threshold"]
    df = pd.DataFrame(columns=columns)
    for i in range(len(class_labels)):
        df.loc[i] = [class_labels[i],
                     round(tp(y[:, i], pred[:, i]),3),
                     round(tn(y[:, i], pred[:, i]),3),
                     round(fp(y[:, i], pred[:, i]),3),
                     round(fn(y[:, i], pred[:, i]),3),
                     round(acc(y[:, i], pred[:, i], thresholds[i]),3),
                     round(prevalence(y[:, i]),3),
                     round(sens(y[:, i], pred[:, i], thresholds[i]),3),
                     round(spec(y[:, i], pred[:, i], thresholds[i]),3),
                     round(ppv(y[:, i], pred[:, i], thresholds[i]),3),
                     round(npv(y[:, i], pred[:, i], thresholds[i]),3),
                     round(auc(y[:, i], pred[:, i]),3),
                     round(f1(y[:, i], pred[:, i] > thresholds[i]),3),
                     round(thresholds[i], 3)]

    df = df.set_index("Injury")
    return df

def bootstrap_metric(y, pred, classes, metric='auc',bootstraps = 100, fold_size = 1000):
    statistics = np.zeros((len(classes), bootstraps))
    if metric=='AUC':
        metric_func = roc_auc_score
    if metric=='Sensitivity':
        metric_func = sensitivity
    if metric=='Specificity':
        metric_func = specificity
    if metric=='Accuracy':
        metric_func = get_accuracy
    for c in range(len(classes)):
        df = pd.DataFrame(columns=['y', 'pred'])
        df.loc[:, 'y'] = y[:, c]
        df.loc[:, 'pred'] = pred[:, c]
        # get positive examples for stratified sampling
        df_pos = df[df.y == 1]
        df_neg = df[df.y == 0]
        prevalence = len(df_pos) / len(df)
        for i in range(bootstraps):
            # stratified sampling of positive and negative examples
            pos_sample = df_pos.sample(n = int(fold_size * prevalence), replace=True)
            neg_sample = df_neg.sample(n = int(fold_size * (1-prevalence)), replace=True)

            y_sample = np.concatenate([pos_sample.y.values, neg_sample.y.values])
            pred_sample = np.concatenate([pos_sample.pred.values, neg_sample.pred.values])
            score = metric_func(y_sample, pred_sample)
            statistics[c][i] = score
    return statistics

def get_confidence_intervals(y,pred,class_labels):
    
    metric_dfs = {}
    for metric in ['AUC','Sensitivity','Specificity','Accuracy']:
        statistics = bootstrap_metric(y,pred,class_labels,metric)
        df = pd.DataFrame(columns=["Mean "+metric+" (CI 5%-95%)"])
        for i in range(len(class_labels)):
            mean = statistics.mean(axis=1)[i]
            max_ = np.quantile(statistics, .95, axis=1)[i]
            min_ = np.quantile(statistics, .05, axis=1)[i]
            df.loc[class_labels[i]] = ["%.2f (%.2f-%.2f)" % (mean, min_, max_)]
        metric_dfs[metric] = df
    return metric_dfs



In [21]:
class_labels = ['ABN','ACL','MEN']
perf_metrics_df = get_performance_metrics(y, pred, class_labels, tp=TP,tn=TN, 
                                          fp=FP, fn=FN, acc=get_accuracy, 
                                          prevalence=get_prevalence, 
                                          spec=specificity,sens=sensitivity, 
                                          ppv=get_ppv, npv=get_npv, 
                                          auc=roc_auc_score, f1=f1_score,
                                          thresholds=[])

In [22]:
get_confidence_intervals(y,pred,['abnormality','acl tear','meniscus tear'])

{'AUC':               Mean AUC (CI 5%-95%)
 abnormality       0.88 (0.87-0.90)
 acl tear          0.81 (0.80-0.83)
 meniscus tear     0.73 (0.71-0.76),
 'Sensitivity':               Mean Sensitivity (CI 5%-95%)
 abnormality               0.97 (0.96-0.98)
 acl tear                  0.91 (0.89-0.93)
 meniscus tear             0.88 (0.85-0.91),
 'Specificity':               Mean Specificity (CI 5%-95%)
 abnormality               0.28 (0.23-0.34)
 acl tear                  0.59 (0.56-0.63)
 meniscus tear             0.40 (0.36-0.43),
 'Accuracy':               Mean Accuracy (CI 5%-95%)
 abnormality            0.83 (0.81-0.84)
 acl tear               0.73 (0.71-0.76)
 meniscus tear          0.61 (0.59-0.63)}

In [23]:
perf_metrics_df

,TP,TN,FP,FN,Accuracy,Prevalence,Sensitivity,Specificity,PPV,NPV,AUC,F1,Threshold
Injury,,,,,,,,,,,,,
ABN,92,7,18,3,0.825,0.792,0.968,0.280,0.836,0.700,0.883,0.898,0.5
ACL,49,39,27,5,0.733,0.450,0.907,0.591,0.645,0.886,0.815,0.754,0.5
MEN,46,27,41,6,0.608,0.433,0.885,0.397,0.529,0.818,0.729,0.662,0.5
